In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb
from sentence_transformers import CrossEncoder

In [2]:
path = "docs/Building_Agentic_AI_Systems_Create_intelligent,_autonomous_AI_agents.pdf"
loader = PyPDFLoader(path)

documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)

chunks = []


In [4]:
for idx, document in enumerate(documents):
    
    chunk_lines = text_splitter.split_text(document.page_content)

    for index, chunk in  enumerate(chunk_lines):
        
        chunks.append({
            "id" : f"page{idx}_chunk{index}",
            "content": chunk,
            "metadata": {
            "title" : "Building Agentic AI Systems Create_intelligent, autonomous AI agents",
            "source": "Building_Agentic_AI_Systems_Create_intelligent,_autonomous_AI_agents.pdf",
            "page": idx
        }})


In [6]:
chunks

[{'id': 'page1_chunk0',
  'content': 'Building Agentic AI Systems\nCreate intelligent, autonomous AI agents that can reason, plan, \nand adapt\nAnjanava Biswas\nWrick Talukdar',
  'metadata': {'title': 'Building Agentic AI Systems Create_intelligent, autonomous AI agents',
   'source': 'Building_Agentic_AI_Systems_Create_intelligent,_autonomous_AI_agents.pdf',
   'page': 1}},
 {'id': 'page2_chunk0',
  'content': 'Building Agentic AI Systems\nCopyright © 2025 Packt Publishing\nAll rights reserved. No part of this book may be reproduced, stored in a retrieval system, or transmitted \nin any form or by any means, without the prior written permission of the publisher, except in the case \nof brief quotations embedded in critical articles or reviews.\nThe author acknowledges the use of cutting-edge AI, such as ChatGPT, with the sole aim of enhancing',
  'metadata': {'title': 'Building Agentic AI Systems Create_intelligent, autonomous AI agents',
   'source': 'Building_Agentic_AI_Systems_Cre

In [3]:

model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

In [15]:
help(chromadb.PersistentClient)

Help on function PersistentClient in module chromadb:

PersistentClient(path: Union[str, pathlib.Path] = './chroma', settings: Optional[chromadb.config.Settings] = None, tenant: str = 'default_tenant', database: str = 'default_database') -> chromadb.api.ClientAPI
    Creates a persistent instance of Chroma that saves to disk. This is useful for
    testing and development, but not recommended for production use.
    
    Args:
        path: The directory to save Chroma's data to. Defaults to "./chroma".
        tenant: The tenant to use for this client. Defaults to the default tenant.
        database: The database to use for this client. Defaults to the default database.



In [4]:
chroma_client = chromadb.PersistentClient(path="./chroma_store")
collection = chroma_client.get_or_create_collection(name="agentic_collection")

In [5]:
collection.count()

1442

In [ ]:
for chunk in chunks:

    collection.add(
        ids=chunk["id"],
        documents=chunk["content"],
        embeddings=model.encode(chunk["content"]),
        metadatas = chunk["metadata"]
    )


In [ ]:
collection.similarty

In [9]:
# test bi encoding

query = "What's the kind of agents ?"
query_embedding = model.encode([query]).tolist()

results = collection.query(
    query_embeddings=query_embedding,
    n_results=2
)

results

{'ids': [['page73_chunk0', 'page155_chunk4']],
 'embeddings': None,
 'documents': [['Questions 49\nQuestions\n1. What are the key characteristics of intelligent agents?\n2. What are the main types of architectural patterns for intelligent agents?\n3. How do deliberative and reactive architectures differ in their strengths and weaknesses?\n4. What is a multi-agent system (MAS) and what are its key characteristics?\n5. What are the main interaction mechanisms in MASs?\n6. In what domains are MASs commonly applied?\nAnswers',
   'while maintaining clear role separation and specialized tool usage for each agent. The following is \nhow you define agents with AutoGen:\n1 flight_agent = AssistantAgent(\n2    name="flight_planner",\n3    model_client=model_client,\n4    tools=[travel_tools.search_flights],']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'page': 73,
    'source': 'Building_Agentic_AI_Systems_Create_intelligent,_autonomous

In [11]:
# cross encoding

cross_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [12]:
import numpy as np

# test bi encoding

query = "governance system"
query_embedding = model.encode([query]).tolist()

candidate_docs = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
    include  = ["documents"]
)["documents"][0]


pairs = [(query, doc) for doc in candidate_docs]


scores = cross_model.predict(pairs)

# Sort candidates by score
reranked = sorted(zip(candidate_docs, scores), key=lambda x: x[1], reverse=True)

print("Final Reranked Results:")
for doc, score in reranked:
    print(f"Score: {score:.4f} | Doc: {doc}")

Final Reranked Results:
Score: 5.4085 | Doc: Understanding self-governance, agency, and autonomy 29
Self-governance
Agentic systems are artificial and human systems that possess self-governance, adaptability, and 
interaction. Self-governance refers to the ability of a system or entity to govern or control itself 
autonomously, without external direction or control. In the context of agentic systems, self-governance 
implies that the system can make its own decisions, set its own goals, and regulate its behavior based
Score: -0.2812 | Doc: on its internal rules, models, and decision-making algorithms. Basically, they operate according to 
their rules and internal states, and execute a change of behavior, if necessary, due to a change of the 
environment or the objectives. Such systems interact with the environment or other systems in a way 
that is meaningful to them and, through these interactions, become influenced.
Some key aspects of self-governance in agentic systems are as follow

In [49]:
candidate_docs

{'ids': [['page73_chunk0',
   'page155_chunk4',
   'page208_chunk4',
   'page109_chunk3',
   'page128_chunk1']],
 'embeddings': None,
 'documents': [['Questions 49\nQuestions\n1. What are the key characteristics of intelligent agents?\n2. What are the main types of architectural patterns for intelligent agents?\n3. How do deliberative and reactive architectures differ in their strengths and weaknesses?\n4. What is a multi-agent system (MAS) and what are its key characteristics?\n5. What are the main interaction mechanisms in MASs?\n6. In what domains are MASs commonly applied?\nAnswers',
   'while maintaining clear role separation and specialized tool usage for each agent. The following is \nhow you define agents with AutoGen:\n1 flight_agent = AssistantAgent(\n2    name="flight_planner",\n3    model_client=model_client,\n4    tools=[travel_tools.search_flights],',
   '1. What are the three primary types of memory architecture in agent systems, and why are they \nimportant for maintain

In [46]:
candidate_docs

{'ids': [['page73_chunk0',
   'page155_chunk4',
   'page208_chunk4',
   'page109_chunk3',
   'page128_chunk1']],
 'embeddings': None,
 'documents': [['Questions 49\nQuestions\n1. What are the key characteristics of intelligent agents?\n2. What are the main types of architectural patterns for intelligent agents?\n3. How do deliberative and reactive architectures differ in their strengths and weaknesses?\n4. What is a multi-agent system (MAS) and what are its key characteristics?\n5. What are the main interaction mechanisms in MASs?\n6. In what domains are MASs commonly applied?\nAnswers',
   'while maintaining clear role separation and specialized tool usage for each agent. The following is \nhow you define agents with AutoGen:\n1 flight_agent = AssistantAgent(\n2    name="flight_planner",\n3    model_client=model_client,\n4    tools=[travel_tools.search_flights],',
   '1. What are the three primary types of memory architecture in agent systems, and why are they \nimportant for maintain

In [6]:
# self-retrieving

from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


document_content_description = (
    "Agentic System Book which explains different archetures of AI systems"
    )

embedding_func = embedding_function = HuggingFaceEmbeddings(
    model_name="distiluse-base-multilingual-cased-v2"
)

vectorstore = Chroma(
    persist_directory="./chroma_store",
    embedding_function=embedding_function,
    collection_name="agentic_collection",
)

metadata_field_info = [
    AttributeInfo(
        name="content",
        description="Chunk content",
        type="string",
    ),
]

In [33]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
load_dotenv()

True

In [8]:
llm_groq = ChatGroq(model = "openai/gpt-oss-20b")

In [9]:
self_retriever = SelfQueryRetriever.from_llm(
    llm_groq,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [25]:
self_retriever.invoke("C'est quoi ReAct")

[Document(id='page134_chunk1', metadata={'title': 'Building Agentic AI Systems Create_intelligent, autonomous AI agents', 'source': 'Building_Agentic_AI_Systems_Create_intelligent,_autonomous_AI_agents.pdf', 'page': 134}, page_content='6    base_url = "<api URL>"\n7\n8    params = {\n9        "q": location,\n10        "appid": API_KEY,\n11       "units": "imperial"  # For Fahrenheit\n12  }\n13  response = requests.get(base_url, params=params)\n14  if response.status_code == 200:\n15      data = response.json()\n16      return data\nAt the minimum, the LLM agent requires the tool’s description of what the tool does and what input \nit expects. Y ou can also specify which parameters (in this case, location and date) are mandatory'),
 Document(id='page108_chunk4', metadata={'source': 'Building_Agentic_AI_Systems_Create_intelligent,_autonomous_AI_agents.pdf', 'page': 108, 'title': 'Building Agentic AI Systems Create_intelligent, autonomous AI agents'}, page_content='and adjust its decision

In [26]:
help(self_retriever)

Help on SelfQueryRetriever in module langchain.retrievers.self_query.base object:

class SelfQueryRetriever(langchain_core.retrievers.BaseRetriever)
 |  SelfQueryRetriever(*args: Any, name: Optional[str] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, vectorstore: langchain_core.vectorstores.base.VectorStore, llm_chain: langchain_core.runnables.base.Runnable[dict, langchain_core.structured_query.StructuredQuery], search_type: str = 'similarity', search_kwargs: dict = <factory>, structured_query_translator: langchain_core.structured_query.Visitor, verbose: bool = False, use_original_query: bool = False) -> None
 |  
 |  Retriever that uses a vector store and an LLM to generate
 |  the vector store queries.
 |  
 |  Method resolution order:
 |      SelfQueryRetriever
 |      langchain_core.retrievers.BaseRetriever
 |      langchain_core.runnables.base.RunnableSerializable[str, list[Document]]
 |      langchain_core.runnables.base.RunnableSerializable


In [23]:
# llm calling

system_cot = """
You are an AI reasoning module. Use the retrieved context and the user query to produce a clear reasoning trace, followed by a final answer.

[CONTEXT]
{{rag_context}}

[QUERY]
{{user_query}}

Follow these steps:
1. Identify key facts from the context relevant to the query.
2. Infer missing links or perform calculations if needed.
3. Produce a short and logically ordered reasoning.
4. Output the final answer in a separate section.

Format:
Reasoning:
<your reasoning steps>

Answer:
<final concise answer>

"""


system_tot = """ 
You are an AI reasoning module. Explore multiple reasoning paths using the retrieved context and the user query.

[CONTEXT]
{{rag_context}}

[QUERY]
{{user_query}}

Follow these steps:
1. Generate 2–3 reasoning branches proposing different interpretations or solution paths.
2. Evaluate each branch for coherence, correctness, and alignment with the context.
3. Select the best branch.
4. Produce the final answer based on that branch.

Format:
Thought Branches:
- Branch 1: <reasoning>
- Branch 2: <reasoning>
- Branch 3 (optional): <reasoning>

Evaluation:
<compare branches and choose the best>

Selected Solution:
<the chosen reasoning>

Answer:
<final concise answer>

"""



In [34]:
prompt_system = PromptTemplate.from_template(system_cot)

In [43]:
from langchain_core.output_parsers import StrOutputParser

p = self_retriever | StrOutputParser()

In [45]:
p.invoke("bonjour")

ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=[Document(id='page275_chu..._content='6   tools=[')], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [40]:
cot_chain = prompt_system  | llm_groq

In [41]:
cot_chain.invoke({"user_query":"what's agentic system", "rag_context":""})

AIMessage(content='I’m ready to help, but I need a bit more information. Could you please provide the context and the specific question you’d like me to address?', additional_kwargs={'reasoning_content': "We need to produce a reasoning trace and answer. But we don't have actual context or query. The placeholders {rag_context} and {user_query} are present. In this scenario, the system didn't provide context or query. Probably the user didn't ask a question. This is a meta prompt? The user didn't provide content. We should respond with an explanation that we need more info. The instruction says we must produce a clear reasoning trace and final answer, but we have no query. So we must indicate that we need context. According to policy, we should ask for clarification."}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 181, 'total_tokens': 338, 'completion_time': 0.159284632, 'prompt_time': 0.015411263, 'queue_time': 0.037530237, 'total_time': 0.174695895, 'co